# Feelpp NIRB Offline

## Import useful modules 

In [ ]:
import sys
import os
### discomment above two line ones to add the feelpp path with nirb module in the python environement
# feelpath = os.path.join('/data/home/elarif/devel/feelpp/build/python/install/lib/python3/dist-packages')
# sys.path.append(feelpath)
# print(sys.path)
from feelpp.mor.nirb.nirb import *
from feelpp.mor.nirb.utils import init_feelpp_environment
import time
import json

## Set configuration

In [ ]:
import IPython
module_locals = IPython.extract_module_locals()[1]
# get directory of the notebook, only working if the notebook is run in vscode
if "__vsc_ipynb_file__" in module_locals:
    model_dir = "/".join(module_locals["__vsc_ipynb_file__"].split("/")[:-1]) + "/model"
# if not, we need to hard code the path...
else:
    model_dir = "/home/saigre/Documents/code/feelpp/python/pyfeelpp-mor/feelpp/mor/nirb/model"

In [ ]:
config_file = os.path.join(model_dir, "square/square.cfg")
N = None # number of initiale snapshot 
greedy_tol = 1.e-5 # convergence criteria of Greedy method
Nmax_greedy = 100
outdir = None 

Set feelpp environment 

In [ ]:
cfg = feelpp.readCfg(config_file)
toolboxType = cfg['nirb']['toolboxType']
e = init_feelpp_environment(toolboxType, config_file)

nirb_file = feelpp.Environment.expand(cfg['nirb']['filename'])
config_nirb = feelpp.readJson(nirb_file)['nirb']

Get number of initial snapshots. By defaults, it will take the value filled in the model path. 

In [ ]:
nbSnap = N
if nbSnap == None:
    nbSnap = config_nirb['nbSnapshots']

Set the output path 

In [ ]:
# doGreedy = config_nirb['greedy-generation']
doGreedy = False
doRectification = config_nirb['doRectification']
rectPath = ["noRect", "Rect"][doRectification]
greedyPath = ["noGreedy", "Greedy"][doGreedy]

if outdir is None:
    RESPATH = f"results/{rectPath}/{greedyPath}"
else:
    RESPATH = outdir

Initialize nirb offline class : either from scratch or from a ToolboxModel already initialized

In [ ]:
generate_from_scratch = False
if generate_from_scratch:
    nirb_off = nirbOffline(initCoarse=True, **config_nirb)
    nirb_off.initModel()

else:
    tb = ToolboxModel(**config_nirb)
    tb.initModel()

    nirb_off = nirbOffline(initCoarse=True, **config_nirb)
    nirb_off.setModel(tb)

In [ ]:
nirb_off.generateOperators(coarse=True)

## Get nirb reduced basis functions 

In [ ]:
Xi_train = None

if doGreedy:
    _, Xi_train, _ = nirb_off.initProblemGreedy(500, greedy_tol, Nmax=Nmax_greedy, Xi_train=Xi_train, computeCoarse=True, samplingMode="log-random")
else:
    Xi_train = nirb_off.initProblem(nbSnap)
nirb_off.generateReducedBasis()

print("Is L2 orthonormalized ?", nirb_off.checkL2Orthonormalized())


## Save datas

In [ ]:
nirb_off.saveData(RESPATH, force=True)

Get infos 

In [ ]:
info = nirb_off.getOfflineInfos()

if feelpp.Environment.isMasterRank():
    print(json.dumps(info, sort_keys=False, indent=4))
    print(f"[NIRB] Offline part Done !")